In [1]:
!pip install -q kaggle pandas matplotlib seaborn langchain-groq langgraph langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [11]:
import os
import glob
os.environ['KAGGLE_USERNAME'] = "yashdoshi8"
os.environ['KAGGLE_KEY'] = "52345c4fd7732ce6cb4c3218d4a1ce88"

In [5]:
!kaggle datasets download -d michaelpawlus/fundraising-data
!kaggle datasets download -d startificial/synthetic-fundraising-opportunities

Dataset URL: https://www.kaggle.com/datasets/michaelpawlus/fundraising-data
License(s): other
  0% 0.00/1.04M [00:00<?, ?B/s]
100% 1.04M/1.04M [00:00<00:00, 748MB/s]
Dataset URL: https://www.kaggle.com/datasets/startificial/synthetic-fundraising-opportunities
License(s): unknown
  0% 0.00/1.12M [00:00<?, ?B/s]
100% 1.12M/1.12M [00:00<00:00, 452MB/s]


In [6]:
import zipfile
with zipfile.ZipFile("fundraising-data.zip", 'r') as zip_ref:
    zip_ref.extractall("./fundraising-data")
with zipfile.ZipFile("synthetic-fundraising-opportunities.zip", 'r') as zip_ref:
    zip_ref.extractall("./synthetic-fundraising-opportunities")

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def load_all_csvs(folder):
    csv_files = glob.glob(folder + "/*.csv")
    dataframes = {}

    for path in csv_files:
        try:
            df = pd.read_csv(path, low_memory=False)
            name = os.path.splitext(os.path.basename(path))[0]
            dataframes[name] = df
            print(f"Loaded → {name} ({df.shape[0]} rows, {df.shape[1]} cols)")
        except Exception as e:
            print(f"Failed loading {path}: {e}")

    return dataframes

real_dfs = load_all_csvs("/content/fundraising-data")
synthetic_dfs = load_all_csvs("/content/synthetic-fundraising-opportunities")

Loaded → data_science_for_fundraising_contact_reports (196 rows, 7 cols)
Loaded → pawlus_small_sample (2878 rows, 8 cols)
Loaded → data_science_for_fundraising_donor_data (34508 rows, 23 cols)
Loaded → data_science_for_fundraising_sample_network (30 rows, 2 cols)
Loaded → generated_data_coded (20000 rows, 15 cols)


In [15]:
donor_df = None
contact_df = None
network_df = None

for name, df in real_dfs.items():
    lname = name.lower()
    if "donor" in lname:
        donor_df = df
    elif "contact" in lname:
        contact_df = df
    elif "network" in lname:
        network_df = df

print("\n--- Identified Tables ---")
print("Donor DF:", donor_df is not None)
print("Contact DF:", contact_df is not None)
print("Network DF:", network_df is not None)


--- Identified Tables ---
Donor DF: True
Contact DF: True
Network DF: True


In [16]:
synthetic_financial_df = None
for name, df in synthetic_dfs.items():
    if "generated" in name or "fundraising" in name or "opportunity" in name:
        synthetic_financial_df = df
        break

print("\nSynthetic Financial DF:", synthetic_financial_df is not None)


Synthetic Financial DF: True


In [17]:
financial_keywords = ["gift", "donation", "amount", "total", "raised", "contribution", "revenue"]

def detect_financial_columns(df):
    if df is None:
        return []
    return [
        col for col in df.columns
        if any(kw in col.lower() for kw in financial_keywords)
    ]

donor_fin_cols = detect_financial_columns(donor_df)
synthetic_fin_cols = detect_financial_columns(synthetic_financial_df)

print("\nDetected Donor Financial Columns:", donor_fin_cols)
print("Detected Synthetic Financial Columns:", synthetic_fin_cols)

# Convert to numeric safely
for col in donor_fin_cols:
    donor_df[col] = pd.to_numeric(donor_df[col], errors="coerce")

for col in synthetic_fin_cols:
    synthetic_financial_df[col] = pd.to_numeric(synthetic_financial_df[col], errors="coerce")



Detected Donor Financial Columns: ['TotalGiving']
Detected Synthetic Financial Columns: ['Amount', 'Donor_Revenue', 'Total_Days', 'Total_Salesforce']


In [18]:
summary_real = donor_df[donor_fin_cols].describe().T if donor_fin_cols else None
summary_synth = synthetic_financial_df[synthetic_fin_cols].describe().T if synthetic_fin_cols else None

print("=== Real Donor Summary ===")
display(summary_real)

print("=== Synthetic Summary ===")
display(summary_synth)

=== Real Donor Summary ===


,count,mean,std,min,25%,50%,75%,max
TotalGiving,34508.0,2363.534195,113801.439902,0.0,0.0,25.0,144.0,12221854.0


=== Synthetic Summary ===


,count,mean,std,min,25%,50%,75%,max
Amount,20000.0,5.008540e+05,2.889243e+05,1017.0,249965.00,501765.5,750154.75,999939.0
Donor_Revenue,20000.0,5.026168e+06,2.883595e+06,10071.0,2540113.25,4998395.5,7541376.25,9999629.0
Total_Days,20000.0,4.992919e+02,2.898866e+02,1.0,249.00,497.0,751.00,999.0
Total_Salesforce,20000.0,5.033109e+02,2.875053e+02,1.0,258.00,503.0,755.00,999.0


In [19]:
os.environ["GROQ_API_KEY"] = "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"

In [21]:
def df_markdown(df):
    try:
        return df.head(10).to_markdown()
    except:
        return "N/A"

GLOBAL_FINANCIAL_SUMMARY = f"""
# Fundraising Financial Summary

## Real Donor Dataset
Detected Columns: {donor_fin_cols}

Summary:
{df_markdown(summary_real)}

## Synthetic Fundraising Opportunities Dataset
Detected Columns: {synthetic_fin_cols}

Summary:
{df_markdown(summary_synth)}
"""

print(GLOBAL_FINANCIAL_SUMMARY[:1000])


# Fundraising Financial Summary

## Real Donor Dataset
Detected Columns: ['TotalGiving']

Summary:
|             |   count |    mean |    std |   min |   25% |   50% |   75% |         max |
|:------------|--------:|--------:|-------:|------:|------:|------:|------:|------------:|
| TotalGiving |   34508 | 2363.53 | 113801 |     0 |     0 |    25 |   144 | 1.22219e+07 |

## Synthetic Fundraising Opportunities Dataset
Detected Columns: ['Amount', 'Donor_Revenue', 'Total_Days', 'Total_Salesforce']

Summary:
|                  |   count |             mean |             std |   min |              25% |             50% |              75% |              max |
|:-----------------|--------:|-----------------:|----------------:|------:|-----------------:|----------------:|-----------------:|-----------------:|
| Amount           |   20000 | 500854           | 288924          |  1017 | 249965           | 501766          | 750155           | 999939           |
| Donor_Revenue    |   20000 |      

In [24]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    )

In [25]:
from typing import List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from pydantic import BaseModel

class FinanceAgentState(BaseModel):
    messages: List[BaseMessage]
    context: str = ""

def profile_node(state: FinanceAgentState):
    return {"context": GLOBAL_FINANCIAL_SUMMARY}

def reasoning_node(state: FinanceAgentState):
    question = state.messages[-1].content
    context = state.context

    prompt = f"""
You are a senior fundraising data analyst.
Use ONLY the dataset summary below to answer the question:

{context}

Question:
{question}

Give a clear, structured analysis.
"""
    response = llm.invoke(prompt)
    return {"messages": state.messages + [AIMessage(content=response.content)]}

# Build graph
graph = StateGraph(FinanceAgentState)
graph.add_node("profile", profile_node)
graph.add_node("reason", reasoning_node)

graph.set_entry_point("profile")
graph.add_edge("profile", "reason")
graph.add_edge("reason", END)

financial_agent = graph.compile()


In [26]:
def ask_financial_agent(q):
    state = FinanceAgentState(messages=[HumanMessage(content=q)])
    out = financial_agent.invoke(state)
    return out["messages"][-1].content


In [27]:
ask_financial_agent("What financial patterns do you observe across both datasets?")

"## Financial Pattern Analysis Across Datasets\n\nTo identify financial patterns across both the Real Donor Dataset and the Synthetic Fundraising Opportunities Dataset, we will examine key statistics from the summary provided.\n\n### Real Donor Dataset Analysis\n\n1. **TotalGiving Distribution**:\n   - The mean TotalGiving is $2,363.53, indicating an average donation amount.\n   - The standard deviation is $113,801, suggesting a wide variability in donation amounts.\n   - The minimum donation is $0, and the maximum is $12,221,900, showing a significant range in donations.\n   - The 25th percentile is $0, the 50th percentile (median) is $25, and the 75th percentile is $144. This indicates that most donations are relatively small, with a long tail of larger donations.\n\n2. **Donation Patterns**:\n   - The high standard deviation and the large difference between the mean and median suggest that the dataset is skewed towards larger donations, which significantly impact the mean.\n   - The

In [28]:
ask_financial_agent("Compare average donation size between the real donor dataset and synthetic dataset.")

"## Comparison of Average Donation Size\n\nTo compare the average donation size between the real donor dataset and the synthetic dataset, we need to analyze the 'TotalGiving' column from the real donor dataset and the 'Amount' column from the synthetic dataset.\n\n### Real Donor Dataset\n\n* The average donation size in the real donor dataset is $2,363.53.\n* This value is calculated from 34,508 data points.\n\n### Synthetic Dataset\n\n* The average donation size in the synthetic dataset is $500,854.\n* This value is calculated from 20,000 data points.\n\n### Comparison\n\n* The average donation size in the synthetic dataset ($500,854) is significantly higher than in the real donor dataset ($2,363.53).\n* This suggests that the synthetic dataset may not be representative of real-world donation patterns, or that it may be skewed towards larger donations.\n* The difference in average donation size between the two datasets is approximately 211 times.\n\n### Conclusion\n\nThe average donat

In [29]:
ask_financial_agent("Segment donors into low, mid, and high tier based on financial metrics available.")

"## Donor Segmentation Analysis\n\nTo segment donors into low, mid, and high tiers based on financial metrics, we will analyze the 'TotalGiving' column from the Real Donor Dataset.\n\n### Summary Statistics\n\nThe 'TotalGiving' column has the following summary statistics:\n- Mean: $2,363.53\n- Standard Deviation: $113,801\n- Minimum: $0\n- 25th Percentile: $0\n- Median (50th Percentile): $25\n- 75th Percentile: $144\n- Maximum: $12,221,900\n\n### Segmentation Approach\n\nWe will use the 25th, 50th, and 75th percentiles to determine the tier boundaries. However, given the highly skewed distribution (mean is significantly higher than the median), we will also consider the absolute values to ensure a reasonable segmentation.\n\n- **Low Tier**: Donors with 'TotalGiving' less than or equal to the 25th percentile ($0) or a small absolute value (e.g., $100). This tier includes a significant portion of donors who have given little to nothing.\n- **Mid Tier**: Donors with 'TotalGiving' between 

In [30]:
ask_financial_agent("Based on the descriptive statistics, what fundraising strategies would you recommend?")

'## Fundraising Strategy Recommendations\n\nBased on the provided descriptive statistics, we can analyze the fundraising data to inform strategy recommendations. We will examine the Real Donor Dataset and the Synthetic Fundraising Opportunities Dataset separately to identify key insights.\n\n### Real Donor Dataset Analysis\n\n1. **Low Median Donations**: The median `TotalGiving` is $25, indicating that most donors contribute relatively small amounts.\n2. **High Standard Deviation**: The high standard deviation ($113,801) suggests that there are some extremely large donations that skew the mean (`$2,363.53`) significantly higher than the median.\n3. **Presence of Zero Donations**: The minimum `TotalGiving` is $0, which may indicate non-donors or lapsed donors in the dataset.\n\n### Synthetic Fundraising Opportunities Dataset Analysis\n\n1. **High Mean Donations**: The mean `Amount` is $500,854, which is significantly higher than the mean `TotalGiving` in the Real Donor Dataset.\n2. **Co